In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

In [40]:
df = pd.read_parquet(r"data/df_kpi_perf_2108_0110.parquet", engine="pyarrow")
config = pd.read_parquet(r"data/df_huawei_config_2108_0110.parquet", engine="pyarrow")

In [41]:
#tirando as horas do df pra poder dar match
df['Data']=df['Data'].dt.date
config['dateday'] = config['dateday'].dt.date
df = df[['eNodeB','Data','TIM_THROU_USER_PDCP_DL (Kbps)', 'TIM_PRB_UTIL_MEAN_DL (%)']]

In [45]:
# Add an index to preserve original order
config['orig_idx'] = config.index
config.head(6)

,neid,LOCALCELLID,dateday,DLEARFCN,cellreselpriority_x,qrxlevmin_x,snonintrasearch,thrshservlow,cellreselpriority_y,qoffsetfreq,qrxlevmin_y,threshxhigh,threshxlow,orig_idx
0,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,NaN,1,0,0
1,18NLITAG01,1,2023-08-21,1542,7,-62,15,0,7,dB-18,NaN,1,0,1
2,18NLITAG01,1,2023-08-21,2800,7,-62,15,0,7,dB10,NaN,1,0,2
3,18NLITAG01,1,2023-08-21,3375,7,-62,15,0,7,dB0,NaN,1,0,3
4,18NLITAG01,1,2023-08-21,9410,7,-62,15,0,5,dB18,NaN,1,0,4
5,18NLITAG01,1,2023-08-21,9510,7,-62,15,0,7,dB0,NaN,1,0,5


In [50]:

merged = config.merge(
    df[['eNodeB', 'Data', 'TIM_THROU_USER_PDCP_DL (Kbps)', 'TIM_PRB_UTIL_MEAN_DL (%)']],
    left_on=['neid', 'dateday'],
    right_on=['eNodeB', 'Data'],
    how='left'
)

# Restore the original order
merged = merged.sort_values('orig_idx')
merged = merged.drop(columns='eNodeB', errors='ignore')  # Remove eNodeB if it exists
merged = merged.drop(columns='Data', errors='ignore')  # Remove orig_idx if

In [ ]:
merged
#eu aumento o número de linhas porque ele faz uma contagem a hora de cada eNodeB, 
# então se ele tiver 10 horas, ele vai ter 10 linhas

,neid,LOCALCELLID,dateday,DLEARFCN,cellreselpriority_x,qrxlevmin_x,snonintrasearch,thrshservlow,cellreselpriority_y,qoffsetfreq,qrxlevmin_y,threshxhigh,threshxlow,orig_idx,TIM_THROU_USER_PDCP_DL (Kbps),TIM_PRB_UTIL_MEAN_DL (%)
0,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,NaN,1,0,0,9455.377060,19.014
1,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,NaN,1,0,0,12700.061449,8.038
2,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,NaN,1,0,0,10109.848984,4.078
3,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,NaN,1,0,0,16529.563141,2.554
4,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,NaN,1,0,0,9299.013731,5.978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4151646,SR-SGON18,11,2023-09-30,3150,7,-60,15,1,6,dB3,-58.0,2,0,266680,NaN,NaN
4151647,SR-SGON18,11,2023-09-30,9410,7,-60,15,1,5,dB12,-58.0,1,0,266681,NaN,NaN
4151648,SR-SGON18,11,2023-10-01,75,7,-60,15,1,7,dB3,-58.0,1,0,266682,NaN,NaN
4151649,SR-SGON18,11,2023-10-01,3150,7,-60,15,1,6,dB3,-58.0,2,0,266683,NaN,NaN


In [48]:
print("orig_idx in config:", 'orig_idx' in config.columns)
print("orig_idx in merged:", 'orig_idx' in merged.columns)
print(merged.columns)
print(merged.head(10))

orig_idx in config: True
orig_idx in merged: True
Index(['neid', 'LOCALCELLID', 'dateday', 'DLEARFCN', 'cellreselpriority_x',
       'qrxlevmin_x', 'snonintrasearch', 'thrshservlow', 'cellreselpriority_y',
       'qoffsetfreq', 'qrxlevmin_y', 'threshxhigh', 'threshxlow', 'orig_idx',
       'eNodeB', 'Data', 'TIM_THROU_USER_PDCP_DL (Kbps)',
       'TIM_PRB_UTIL_MEAN_DL (%)'],
      dtype='object')
         neid  LOCALCELLID     dateday  DLEARFCN  cellreselpriority_x  \
0  18NLITAG01            1  2023-08-21       100                    7   
1  18NLITAG01            1  2023-08-21       100                    7   
2  18NLITAG01            1  2023-08-21       100                    7   
3  18NLITAG01            1  2023-08-21       100                    7   
4  18NLITAG01            1  2023-08-21       100                    7   
5  18NLITAG01            1  2023-08-21       100                    7   
6  18NLITAG01            1  2023-08-21       100                    7   
7  18NLITAG01   